In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('data.csv')

In [3]:
columns = ['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Engine Cylinders', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP', 'Transmission Type']
df = df[columns]
df

,Make,Model,Year,Engine HP,Engine Cylinders,Engine Cylinders,Vehicle Style,highway MPG,city mpg,MSRP,Transmission Type
0,BMW,1 Series M,2011,335.0,6.0,6.0,Coupe,26,19,46135,MANUAL
1,BMW,1 Series,2011,300.0,6.0,6.0,Convertible,28,19,40650,MANUAL
2,BMW,1 Series,2011,300.0,6.0,6.0,Coupe,28,20,36350,MANUAL
3,BMW,1 Series,2011,230.0,6.0,6.0,Coupe,28,18,29450,MANUAL
4,BMW,1 Series,2011,230.0,6.0,6.0,Convertible,28,18,34500,MANUAL
...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,6.0,4dr Hatchback,23,16,46120,AUTOMATIC
11910,Acura,ZDX,2012,300.0,6.0,6.0,4dr Hatchback,23,16,56670,AUTOMATIC
11911,Acura,ZDX,2012,300.0,6.0,6.0,4dr Hatchback,23,16,50620,AUTOMATIC
11912,Acura,ZDX,2013,300.0,6.0,6.0,4dr Hatchback,23,16,50920,AUTOMATIC


In [4]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.fillna(0)
df = df.rename(columns={'msrp': 'price'})

# Question 1

In [5]:
# df.groupby('transmission_type').count()
print('q1:', df['transmission_type'].value_counts().idxmax())

q1: AUTOMATIC


# Question 2

In [6]:
df.head(5).T

,0,1,2,3,4
make,BMW,BMW,BMW,BMW,BMW
model,1 Series M,1 Series,1 Series,1 Series,1 Series
year,2011,2011,2011,2011,2011
engine_hp,335.0,300.0,300.0,230.0,230.0
engine_cylinders,6.0,6.0,6.0,6.0,6.0
engine_cylinders,6.0,6.0,6.0,6.0,6.0
vehicle_style,Coupe,Convertible,Coupe,Coupe,Convertible
highway_mpg,26,28,28,28,28
city_mpg,19,19,20,18,18
price,46135,40650,36350,29450,34500


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               11914 non-null  object 
 1   model              11914 non-null  object 
 2   year               11914 non-null  int64  
 3   engine_hp          11845 non-null  float64
 4   engine_cylinders   11884 non-null  float64
 5   engine_cylinders   11884 non-null  float64
 6   vehicle_style      11914 non-null  object 
 7   highway_mpg        11914 non-null  int64  
 8   city_mpg           11914 non-null  int64  
 9   price              11914 non-null  int64  
 10  transmission_type  11914 non-null  object 
dtypes: float64(3), int64(4), object(4)
memory usage: 1024.0+ KB


In [8]:
numerical_columns = ['engine_hp', 'engine_cylinders', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'year']
categorical_columns = ['make', 'model', 'vehicle_style', 'transmission_type']

In [9]:
df[numerical_columns].corrwith(df.price).sort_values(ascending=False)

engine_hp           0.662008
engine_cylinders    0.531312
engine_cylinders    0.531312
engine_cylinders    0.531312
engine_cylinders    0.531312
year                0.227590
city_mpg           -0.157676
highway_mpg        -0.160043
dtype: float64

In [10]:
print('q2: engine_hp and engine_cylinders')

q2: engine_hp and engine_cylinders


# Question 3

In [11]:
average = df.price.mean()
df['above_average'] = df['price'] >average

In [12]:
from sklearn.model_selection import train_test_split
seed = 42

In [13]:
df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=seed)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=seed)

y_train = df_train.above_average
y_valid = df_valid.above_average
y_test = df_test.above_average

df_train.drop(columns=['above_average', 'price'], inplace = True)
df_valid.drop(columns=['above_average', 'price'], inplace = True)
df_test.drop(columns=['above_average', 'price'], inplace = True)

In [14]:
from sklearn.metrics import mutual_info_score

In [15]:
def calculate_mutual_info(x, y):
    return mutual_info_score(x, y)

df_train_mi = df_train[categorical_columns].apply(lambda x: calculate_mutual_info(x, y_train))

In [16]:
print('q3: ', df_train_mi.idxmin())

q3:  transmission_type
